In [1]:
import findspark
findspark.init('/home/mojtaba/spark-2.1.0-bin-hadoop2.7')

In [2]:
import os
data_dir = os.path.join(os.getcwd(), 'data')

In [3]:
import pyspark

If you have worked with pandas or R dataframes, this should be very similar. In psark 2.0 and higher, spark provides the dataframe syntax.
    - MLLib library API for machine learning
    - 

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [6]:
df = spark.read.json(os.path.join(data_dir, 'people.json'))

In [7]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [8]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [9]:
df.columns

['age', 'name']

See a statistical summary of  numeric columns:

In [10]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



# Manually set your df schema:

In [11]:
from pyspark.sql.types import (StructField, StringType,
                               IntegerType, ArrayType, StructType)

In [12]:
data_schema = [
    StructField('age', IntegerType(), True),
#                name, type,         nullable?
    StructField('name', StringType(), True),
]

In [13]:
final_struct = StructType(fields=data_schema)

Specifying schema when reading:

In [14]:
df = spark.read.json(os.path.join(data_dir, 'people.json'), schema=final_struct)

In [15]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



Now age is integer instead of long.

# Part 2

## Getting a column

In [16]:
df['age']

Column<b'age'>

In [17]:
type(df['age'])

pyspark.sql.column.Column

We have touse the select method, this now returns a dataframe that contains a single column. This gives more flexibility.

In [18]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



If we want to check the row object, use the following:

In [19]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

Returns a list of rows.

In [20]:
df.head(2)[0]

Row(age=None, name='Michael')

In [21]:
type(df.head(2)[0])

pyspark.sql.types.Row

Selecting multiple columns:

In [22]:
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



Returns a new dataframe, This is not an inplace operation. 

In [23]:
df.withColumn('new_age', df['age'] * 2).show()

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     60|
|  19| Justin|     38|
+----+-------+-------+



In [24]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [25]:
df.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



Again, these changes are not to the original dataframe:

In [26]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Using pure SQL

In [27]:
df.createOrReplaceTempView('people')

In [28]:
results = spark.sql('SELECT * FROM people WHERE age >= 20')

In [29]:
results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

